In [229]:

import random
import hashlib
import base64
import numpy as np

## $\text{Parte I}$

### Funções utilitárias

In [200]:
Key = tuple[int, int]

def is_prime(n):
    """Teste de primalidade de Miller-Rabin. Detalhes: https://pt.wikipedia.org/wiki/Teste_de_primalidade_de_Miller%E2%80%93Rabin"""
    # 1 < a < n
    a = random.randint(2, n - 1)

    d = 0
    # s = max{r ∈ N | 2^r divide n - 1}
    for r in range(1, n):
        if (n - 1) % (2 ** r) == 0:
            # d = (n - 1) / (2^r)
            d = (n - 1) // (2 ** r)
        else:
            break
        
    # a^d ≡ 1 mod n
    if pow(a, d, n) == 1:
        return True
    else:
        return False


def generate_big_prime_pair(b):
    p = random.getrandbits(b)
    q = random.getrandbits(b)

    while not is_prime(p):
        p = random.getrandbits(b)
    
    while not is_prime(q):
        q = random.getrandbits(b)

    return (p, q)   

def I2OSP(x: int, l: int):
    """Integer-to-Octet-String, responsável por converter um inteiro em uma string de bytes (octeto).
       Detalhes:"https://www.inf.pucrs.br/calazans/graduate/TPVLSI_I/RSA-oaep_spec.pdf, seção 1.1.2"""
    if x >= 256 ** l:
        raise ValueError("Inteiro grande demais.")
    
    # conversão para base 256 com tamanho l
    return x.to_bytes(l, byteorder='big')

def OS2IP(X: bytes):
    """Octet-String-to-Integer, responsável por converter uma string de bytes (octeto) em um inteiro
       Detalhes:"https://www.inf.pucrs.br/calazans/graduate/TPVLSI_I/RSA-oaep_spec.pdf, seção 1.1.2"""
    return int.from_bytes(X, byteorder='big')

def bitwise_xor(a: bytes, b: bytes):
    """Operação XOR entre dois bytes"""
    r=b""
    
    for i in range(max(len(a), len(b))):
        if i >= len(a):
            r += b[i].to_bytes(1, byteorder='big')
        elif i >= len(b):
            r += a[i].to_bytes(1, byteorder='big')
        else:
            r += (a[i] ^ b[i]).to_bytes(1, byteorder='big')
    
    return r
            

def get_key_len(key: Key):
    """Retorna o número de octetos no modulo n da chave"""
    _, n = key
    return n // 8

### Geração de chaves

In [219]:

def generate_public_key(phi: int):
    # A chave pública e é um número primo tal que 1 < e < φ(n) e mdc(e, φ(n)) = 1
    e = random.randint(2, phi - 1)
    
    while not is_prime(e):
        e = random.randint(2, phi - 1)

    return e

def generate_private_key(e: int, phi: int):
    # A chave privada d é um número tal que e*d ≡ 1 mod φ(n)
    d = pow(e, -1, phi)
    
    return d

def generate_keypair() -> tuple[Key, Key]:
    p, q = generate_big_prime_pair(1024)
    
    print(f"p: {p}")
    print(f"q: {q}")
    
    n = p * q
    phi = (p - 1) * (q - 1)
    
    e = generate_public_key(phi)
    d = generate_private_key(e, phi)

    return ((e, n), (d, n))

### RSA

In [206]:
def RSA_encode(key: Key, m: int):
    e, n = key

    if m >= get_key_len(key):
        raise ValueError("Mensagem maior ou igual ao módulo.")

    return pow(m, e, n)

def RSA_decode(key: Key, c: int):
    d, n = key
    
    return pow(c, d, n)

### Padding e criptografia e descriptografia
Detalhes: https://www.inf.pucrs.br/calazans/graduate/TPVLSI_I/RSA-oaep_spec.pdf, seção 1.3.

In [233]:

def MGF(Z: bytes, l: int):
    """MGF utilizando o Hash SHA-3"""
    
    # bLen denota o comprimento nos octetos da saída da função hash
    bLen = hashlib.sha3_256().digest_size
    
    if l > 2**32 * bLen:
        raise ValueError("Máscara muito longa.")
    
    T = b""
    
    for i in range(Math.ceil(l / bLen)):
        # Converte i em um octeto C de tamanho 4 com a primitiva I2OSP.
        C = I2OSP(i, 4)
        # Concatena o resultado do hash SHA-3 de seed Z e C com T.
        T += hashlib.sha3_256(Z+C).digest()
    
    # A máscara M é a string consistindo dos primeiros l octetos de T.
    return T[:l]

def OAEP_encode(M: bytes, emLen: int, P: bytes = b""):
    """Encode de OAEP utilizando o Hash SHA-3"""
    if len(P) > 2**64 - 1:
        raise ValueError("Comprimento de P muito grande.")

    bLen = hashlib.sha3_256().digest_size
    mLen = len(M)

    if mLen > emLen - 2 * bLen - 1:
        raise ValueError("Mensagem muito grande.")
    
    # Geração de uma string de octetos de comprimento (emLen - mLen - 2 * bLen - 1) consistindo de zeros.
    # Erro de overflow: "cannot fit 'int' into an index-sized integer"
    chunk_size = 10**6  # Tamanho do chunk (ajustável)
    PS = bytearray()
    PS_len = emLen - mLen - 2 * bLen - 1

    for _ in range(0, PS_len, chunk_size):
        PS.extend(b"\x00" * min(chunk_size, PS_len - len(PS)))

    PS = bytes(PS)  # Converte para bytes, se necessário
    
    pHash = hashlib.sha3_256(P).digest()
    
    # Concatenação de pHash, PS, um octeto 0x01 e a mensagem M no bloco de dados DB.
    DB = pHash + PS + b"\x01" + M
    
    seed = random.getrandbits(bLen * 8).to_bytes(bLen, byteorder='big')
    
    # Mascaramento do bloco de dados
    dbMask = MGF(seed, emLen - bLen)
    maskedDB = bitwise_xor(DB, dbMask)

    # Mascaramento da seed
    seedMask = MGF(maskedDB, bLen)        
    maskedSeed = bitwise_xor(seed, seedMask)
    
    return maskedSeed + maskedDB

def OAEP_decode(EM: bytes, P: bytes = b""):
    """Decode de OAEP utilizando o Hash SHA-3"""
    bLen = hashlib.sha3_256().digest_size
    emLen = len(EM)
    
    if len(P) > 2**64 - 1 or len(EM) < 2 * bLen + 1:
        raise ValueError("Erro na decodificação. Tamanho de P ou EM inválido.")
    
    maskedSeed = EM[:bLen]
    maskedDB = EM[bLen:]
    
    seedMask = MGF(maskedDB, bLen)
    seed = bitwise_xor(maskedSeed, seedMask)
    
    dbMask = MGF(seed, emLen - bLen)
    DB = bitwise_xor(maskedDB, dbMask)
    
    pHash = hashlib.sha3_256(P).digest()
    
    # Separa a mensagem supondo o formato pHash_ || PS || 01 || M
    pHash_ = DB[:bLen]        
        
    if pHash != pHash_:
        raise ValueError("Erro na decodifiocação.")
    
    # Busca o byte 01 após o padding.
    i = bLen
    while DB[i] == 0:
        i += 1
    
    if DB[i] != 1:
        raise ValueError("Byte 0x01 não encontrado.")
    
    return DB[i+1:]


### Criptografia e descriptografia

In [221]:
def encrypt(key: Key, M: bytes, P: bytes = b""):
    """"Criptografa a mensagem usando RSA-OAEP"""
    try:
        e, n = key
        
        emLen = get_key_len(key)
        
        EM = OAEP_encode(M, emLen, P)

        c = RSA_encode(key, EM)
        
        C = I2OSP(c, emLen)
        
        return C
    
    except ValueError as e:
        print(f"Erro: {e}")
        return None
    
def decrypt(key: Key,  C: bytes, P: bytes = b""):
    """"Descriptografa a mensagem usando RSA-OAEP"""
    try:
        emLen = get_key_len(key)
        
        c = OS2IP(C)
        
        m = RSA_decode(key, c)
        
        EM = I2OSP(m, emLen)
        
        M = OAEP_decode(EM, P)
        
        return M
    
    except ValueError as e:
        print(f"Erro: {e}")
        return None

    

## Teste

In [ ]:
public_key, private_key = generate_keypair()

M = b"Foo"

C = encrypt(public_key, M)

M_ = decrypt(private_key, C)

print(f"Mensagem original: {M}")
print(f"Mensagem decifrada: {M_}")

if M == M_:
    print("Sucesso!")
else:
    print("Falha!")

p: 161958163785256730231737186645503577112684047161655052206626919799460739878857885971240376040739317127864766412694217927234503865624117508225387097958149603936969347025398911971314951697531546982033645741685751518147360643287127147058248080980039847813272539165705540331566391306450791929122565456873815157712
q: 138540468492144722107282709408609691372167251407473606128004444961067779323478953691825543180844480481421199648327618884639875922601084397053865439683552957018378993480108045774444889197742217205651545768766756392468587997656076257312673257722239224315175292911322927995284690657983189415533391920114135084174
